In [0]:
from pyspark.sql.functions import *

In [0]:
df= spark.read.options(header=True,sep='|').csv("dbfs:/FileStore/final.csv")

In [0]:
df1=df.select("*",split(df['data'],",").alias('temp'))
df1.printSchema()
df1.show(3)

root
 |-- _c0: string (nullable = true)
 |-- city_name: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- data: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lon: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- timezone: string (nullable = true)
 |-- temp: array (nullable = true)
 |    |-- element: string (containsNull = false)

+---+---------+------------+--------------------+----+-----+----------+----------------+--------------------+
|_c0|city_name|country_code|                data| lat|  lon|state_code|        timezone|                temp|
+---+---------+------------+--------------------+----+-----+----------+----------------+--------------------+
|  0|Four Oaks|          US|{'precip': 0, 'sn...|35.5|-78.5|        NC|America/New_York|[{'precip': 0,  '...|
|  1|Four Oaks|          US|{'precip': 0, 'sn...|35.5|-78.5|        NC|America/New_York|[{'precip': 0,  '...|
|  2|Four Oaks|          US|{'precip': 0, 'sn...|35.

In [0]:
df2=df1.withColumn("precip",df1.temp[0])\
    .withColumn("snow",df1.temp[1])\
        .withColumn("tempreture",df1.temp[2])\
            .withColumn("timestamp_local",df1.temp[3])\
                .withColumn("timestamp_global",df1.temp[4])

In [0]:
df3=df2.drop('temp','data')
df3.show(2)

+---+---------+------------+----+-----+----------+----------------+------------+----------+-------------+--------------------+--------------------+
|_c0|city_name|country_code| lat|  lon|state_code|        timezone|      precip|      snow|   tempreture|     timestamp_local|    timestamp_global|
+---+---------+------------+----+-----+----------+----------------+------------+----------+-------------+--------------------+--------------------+
|  0|Four Oaks|          US|35.5|-78.5|        NC|America/New_York|{'precip': 0| 'snow': 0| 'temp': 10.9| 'timestamp_local...| 'timestamp_utc':...|
|  1|Four Oaks|          US|35.5|-78.5|        NC|America/New_York|{'precip': 0| 'snow': 0| 'temp': 10.9| 'timestamp_local...| 'timestamp_utc':...|
+---+---------+------------+----+-----+----------+----------------+------------+----------+-------------+--------------------+--------------------+
only showing top 2 rows



In [0]:
df3=df3.select("*",split(df3['precip'],':').alias('precip1'),\
    split(df3['tempreture'],':').alias('tempreture1'),\
        split(df3['snow'],':').alias('snow1'),\
            split(df3['timestamp_local'],':').alias('timestamplocal1'),\
                split(df3['timestamp_global'],':').alias('timestampglobal1')\
                )
df3=df3.drop('precip','snow','tempreture','timestamp_local','timestamp_global')                

In [0]:
df4=df3.withColumn('precipitate',df3.precip1[1])\
    .withColumn('tempreture',df3.tempreture1[1])\
        .withColumn('snow',df3.snow1[1])\
            .withColumn('timestamp_local',df3.timestamplocal1[1])\
                .withColumn('timestamp_global',df3.timestampglobal1[1])

               
df5=df4.drop('precip1','snow1','tempreture1','timestamplocal1','timestampglobal1')


In [0]:
df5.show(6)

+---+---------+------------+----+-----+----------+----------------+-----------+----------+----+---------------+----------------+
|_c0|city_name|country_code| lat|  lon|state_code|        timezone|precipitate|tempreture|snow|timestamp_local|timestamp_global|
+---+---------+------------+----+-----+----------+----------------+-----------+----------+----+---------------+----------------+
|  0|Four Oaks|          US|35.5|-78.5|        NC|America/New_York|          0|      10.9|   0| '2024-03-21T04|  '2024-03-21T08|
|  1|Four Oaks|          US|35.5|-78.5|        NC|America/New_York|          0|      10.9|   0| '2024-03-21T04|  '2024-03-21T08|
|  2|Four Oaks|          US|35.5|-78.5|        NC|America/New_York|          0|      10.9|   0| '2024-03-21T04|  '2024-03-21T08|
|  3|Four Oaks|          US|35.5|-78.5|        NC|America/New_York|          0|      10.9|   0| '2024-03-21T04|  '2024-03-21T08|
|  4|Four Oaks|          US|35.5|-78.5|        NC|America/New_York|          0|      10.9|   0| '

In [0]:
df5.write.options(Header=True).csv('dbfs:/FileStore/output/tempdata.csv')